In [ ]:
from skimage import io
import pandas as pd
import numpy as np
import os
import random

In [ ]:
# generate labels for conditional GAN
def cgan(folder='cgan',home='problem2',resize=1.0,decay=0.005,interval=100,
         formatting=['d_0','power_0','speed_0',
                 'd_1','power_1','speed_1',
                 'd_2','power_2','speed_2',
                 'd_3','power_3','speed_3',
                 'd_4','power_4','speed_4',
                 'd_5','power_5','speed_5']):
    # create directory
    if not os.path.exists('./%s'%(folder)):os.mkdir('./%s'%(folder))
    df=[]
    for num in range(1,251):
        print('Layer: %d'%(num))
        layer='L%04d'%(num)
        cmd=pd.read_csv('./%s/Command_Part1/XYPT_Part01_%s.csv'%(home,layer),header=None)
        left=cmd[0].min()
        right=cmd[0].max()
        down=cmd[1].min()
        up=cmd[1].max()
        mag=1000
        points={}
        locations={}
        current_order=0
        first_row=cmd[0:1]
        history_x=[first_row[0][0]]*3
        history_y=[first_row[1][0]]*3
        speed_max=-10000
        speed_min=10000
        power_max=-10000
        power_min=10000
        for line in cmd.iterrows():
            x=line[1][0]
            y=line[1][1]
            power=line[1][2]
            history_x.pop()
            history_x.insert(0,x)
            history_y.pop()
            history_y.insert(0,y)
            speed=((history_x[2]-history_x[0])*(history_x[2]-history_x[0])+(history_y[2]-history_y[0])*(history_y[2]-history_y[0]))**0.5/2e-5
            speed_max=max(speed,speed_max)
            speed_min=min(speed,speed_min)
            power_max=max(power,power_max)
            power_min=min(power,power_min)
            points[current_order]={'x':int((x-left)*mag),'y':int((y-down)*mag),'power':power,'speed':speed}
            if line[1][3]==2: locations[len(locations)+1]=current_order
            current_order+=1
        for temp in locations:
            if temp%100==0: print(temp,end='\t')
            row=['./%s/MPM/Layer%03dto%03d/%s/frame_%04d.bmp'%(home,(num-1)//50*50+1,(num-1)//50*50+50,layer,temp)]
            center=points[locations[temp]]
            for data_point in formatting:
                col,idx=data_point.split('_')
                current_idx=locations[temp]-int(idx)*interval
                if current_idx not in points: continue
                current=points[current_idx]
                if col=='x':
                    value=(current['x']-center['x'])*resize/mag
                elif col=='y':
                    value=(current['y']-center['y'])*resize/mag
                elif col=='d':
                    value=((current['x']-center['x'])*(current['x']-center['x'])+(current['y']-center['y'])*(current['y']-center['y']))**0.5*resize/mag
                elif col=='power':
                    value=(current['power']-power_min)/(power_max-power_min)*np.exp(-int(idx)*interval*decay)
                elif col=='speed':
                    value=(current['speed']-speed_min)/(speed_max-speed_min)
                row.append(value)
            df.append(row)
        print('Done')
    pd.DataFrame(df,columns=['path']+formatting).to_csv('./%s/vec.csv'%(folder),index=0)

In [ ]:
cgan()